In [8]:
%load_ext autoreload
%autoreload 2


from glob import glob
from itertools import chain

import ensembl_rest
import pandas as pd
from sourmash import load_signatures


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
cd /home/olga/pureScratch/kmer-hashing/nextflow-intermediates/15/8d1f6b72d37d21de30e1e2249c0e62

/mnt/pureScratch/olga/kmer-hashing/nextflow-intermediates/15/8d1f6b72d37d21de30e1e2249c0e62


In [4]:
ls -lha

total 97K
drwxrwxr-x 1 olga olga    0 Aug  5 14:53 ./
drwxrwxr-x 1 olga olga    0 Aug  5 09:57 ../
-rw-rw-r-- 1 olga olga    0 Aug  5 13:57 .command.begin
-rw-rw-r-- 1 olga olga  43K Aug  5 13:58 .command.err
-rw-rw-r-- 1 olga olga  43K Aug  5 13:58 .command.log
-rw-rw-r-- 1 olga olga    0 Aug  5 13:57 .command.out
-rw-rw-r-- 1 olga olga 9.8K Aug  5 13:57 .command.run
-rw-rw-r-- 1 olga olga  189 Aug  5 13:57 .command.sh
-rw-r--r-- 1 root root    0 Aug  5 13:57 .command.trace
-rw-rw-r-- 1 olga olga    3 Aug  5 14:53 .exitcode
lrwxrwxrwx 1 olga olga  169 Aug  5 13:57 gencode.v30.pc_translations.fa_molecule-protein_ksize-51_log2sketchsize-5.sig -> /mnt/pureScratch/olga/kmer-hashing/nextflow-intermediates/a2/2db47d056a1be7d1d6157fb705be1f/gencode.v30.pc_translations.fa_molecule-protein_ksize-51_log2sketchsize-5.sig
lrwxrwxrwx 1 olga olga  170 Aug  5 13:57 gencode.vM21.pc_translations.fa_molecule-protein_ksize-51_log2sketchsize-5.sig -> /mnt/pureScratch/olga/kmer-hashing/nextflow-intermedia

In [5]:
filenames = glob("*.sig")


In [26]:
loaded_signatures = chain(*[load_signatures(f) for f in filenames])

name_to_signature = {x.name(): x for x in loaded_signatures}

...sig loading 99,730
...sig loading 66,649


In [52]:
len(name_to_signature)

166381

In [55]:
from collections import Counter

protein_id_counter = Counter(key.split("|")[0].split(".")[0] for key, value in name_to_signature.items())
protein_ids = pd.Series(protein_id_counter)
protein_ids.head()

ENSP00000493376    1
ENSP00000334393    1
ENSP00000409316    1
ENSP00000329982    1
ENSP00000411579    1
dtype: int64

In [62]:
protein_ids[protein_ids > 1]

ENSP00000381976    2
ENSP00000394848    2
ENSP00000412018    2
ENSP00000400904    2
ENSP00000371073    2
ENSP00000371079    2
ENSP00000408558    2
ENSP00000399510    2
ENSP00000405307    2
ENSP00000414181    2
ENSP00000316598    2
ENSP00000375080    2
ENSP00000371038    2
ENSP00000370990    2
ENSP00000335505    2
ENSP00000370987    2
ENSP00000370978    2
ENSP00000383641    2
ENSP00000485269    2
ENSP00000370979    2
ENSP00000370940    2
ENSP00000440491    2
ENSP00000394227    2
ENSP00000370935    2
ENSP00000410667    2
ENSP00000416437    2
ENSP00000370920    2
ENSP00000476684    2
ENSP00000348058    2
ENSP00000436825    2
                  ..
ENSP00000370639    2
ENSP00000370627    2
ENSP00000370631    2
ENSP00000392053    2
ENSP00000334113    2
ENSP00000391778    2
ENSP00000402741    2
ENSP00000403045    2
ENSP00000420788    2
ENSP00000370620    2
ENSP00000370621    2
ENSP00000498725    2
ENSP00000370616    2
ENSP00000419148    2
ENSP00000479999    2
ENSP00000370582    2
ENSP000003705

In [61]:
len(name_to_signature) - len(protein_ids)

74

In [56]:
protein_id_counter.most_common(10)

[('ENSP00000381976', 2),
 ('ENSP00000394848', 2),
 ('ENSP00000412018', 2),
 ('ENSP00000400904', 2),
 ('ENSP00000371073', 2),
 ('ENSP00000371079', 2),
 ('ENSP00000408558', 2),
 ('ENSP00000399510', 2),
 ('ENSP00000405307', 2),
 ('ENSP00000414181', 2)]

In [60]:
[x for x in name_to_signature.keys() if 'ENSP00000371079' in x]

['ENSP00000371079.3|ENST00000381663.8|ENSG00000182378.14|OTTHUMG00000022693.6|-|PLCXD1-202|PLCXD1|323',
 'ENSP00000371079.3|ENST00000381663.8_PAR_Y|ENSG00000182378.14_PAR_Y|OTTHUMG00000022693.6|-|PLCXD1-202|PLCXD1|323']

In [63]:
ensembl_rest.lookup("ENSP00000371079")

{'db_type': 'core',
 'length': 323,
 'id': 'ENSP00000371079',
 'species': 'homo_sapiens',
 'start': 284188,
 'object_type': 'Translation',
 'Parent': 'ENST00000381663',
 'end': 299335}

In [54]:
protein_id_to_signature = {key.split("|")[0].split(".")[0]: value for key, value in name_to_signature.items()}
len(protein_id_to_signature)

166307

In [46]:
gene_id_to_signature = {key.split("|")[2].split(".")[0]: value for key, value in name_to_signature.items()}

In [51]:
len(gene_id_to_signature)

42850

In [45]:
for i, key in zip(range(10), name_to_signature.keys()):
    print(key)
#     print(ensembl_rest.homology_ensemblgene(key, target_species='mus_musuclus'))

ENSP00000493376.2|ENST00000641515.2|ENSG00000186092.6|OTTHUMG00000001094.4|OTTHUMT00000003223.4|OR4F5-202|OR4F5|326
ENSP00000334393.3|ENST00000335137.4|ENSG00000186092.6|OTTHUMG00000001094.4|-|OR4F5-201|OR4F5|305
ENSP00000409316.1|ENST00000426406.3|ENSG00000284733.1|OTTHUMG00000002860.3|OTTHUMT00000007999.3|OR4F29-201|OR4F29|312
ENSP00000329982.2|ENST00000332831.4|ENSG00000284662.1|OTTHUMG00000002581.3|OTTHUMT00000007334.3|OR4F16-201|OR4F16|312
ENSP00000411579.2|ENST00000420190.6|ENSG00000187634.12|OTTHUMG00000040719.11|OTTHUMT00000316521.3|SAMD11-203|SAMD11|358
ENSP00000393181.1|ENST00000437963.5|ENSG00000187634.12|OTTHUMG00000040719.11|OTTHUMT00000097862.5|SAMD11-204|SAMD11|109
ENSP00000342313.3|ENST00000342066.8|ENSG00000187634.12|OTTHUMG00000040719.11|OTTHUMT00000276866.3|SAMD11-202|SAMD11|681
ENSP00000480870.1|ENST00000618181.4|ENSG00000187634.12|OTTHUMG00000040719.11|-|SAMD11-213|SAMD11|556
ENSP00000482138.1|ENST00000622503.4|ENSG00000187634.12|OTTHUMG00000040719.11|-|SAMD11-217|

In [27]:
ensembl_rest.lookup("ENSP00000493376")

{'species': 'homo_sapiens',
 'start': 65565,
 'end': 70008,
 'Parent': 'ENST00000641515',
 'object_type': 'Translation',
 'db_type': 'core',
 'id': 'ENSP00000493376',
 'length': 326}

In [64]:
for i, (gene_id, sig) in zip(range(2), gene_id_to_signature.items()):
    print('----', gene_id, '----')
    ensembl_data = ensembl_rest.homology_ensemblgene(gene_id, target_species='mus_musuclus')
    

---- ENSG00000186092 ----
---- ENSG00000284733 ----


In [71]:
data = ensembl_data['data']
len(data)

1

In [73]:
data[0].keys()

dict_keys(['homologies', 'id'])

In [74]:
data[0]['id']

'ENSG00000284733'

In [76]:
homologies = data[0]['homologies']
len(homologies)

215

In [77]:
homology = homologies[0]
homology

{'method_link_type': 'ENSEMBL_PARALOGUES',
 'target': {'protein_id': 'ENSP00000467301',
  'species': 'homo_sapiens',
  'align_seq': '-------MVTEFIFLGLSDSQGLQTFLFMLFFVFYGGIVFGNLLIVITVVSDSHLHSPMYFLLANLSLIDLSLSSVTAPKMITDFFSQRKVISFKGCLVQIFLLHFFGGSEMVILIAMGFDRYIAICKPLHYTTIMCGNACVGIMAVAWGIGFLHSVSQLAFAVHLPFCGPNEVDSFYCDLPRVIKLACTDTYRLDIMVIANSGVLTVCSFVLLIISYTIILMTIQHRPLD-KSSKALSTLTAHITVVLLFFGPCVFIYAWPFP-IKSLDKFLAVFYSVITPLLNPIIYTLRNKDMKTAIRQL-RKWDA--HSSVKF--',
  'cigar_line': '7D224MD32MD38MD5M2D6M2D',
  'perc_pos': 72.7869,
  'taxon_id': 9606,
  'id': 'ENSG00000176695',
  'perc_id': 57.0492},
 'type': 'other_paralog',
 'dn_ds': 0.1908,
 'taxonomy_level': 'Vertebrata',
 'source': {'protein_id': 'ENSP00000409316',
  'species': 'homo_sapiens',
  'perc_pos': 71.1538,
  'cigar_line': '228MD37MD39M2D3MD2MDMD2M',
  'align_seq': 'MDGENHSVVSEFLFLGLTHSWEIQLLLLVFSSVLYVASITGNILIVFSVTTDPHLHSPMYFLLASLSFIDLGACSVTSPKMIYDLFRKRKVISFGGCIAQIFFIHVVGGVEMVLLIAMAFDRYVALCKPLHYLTIMSPRMCLSFLAVAWTLGVSHSLFQLAFLVNLAFCGPNVLD

In [86]:
print(homology['target']['align_seq'][:100])
print(homology['source']['align_seq'][:100])

-------MVTEFIFLGLSDSQGLQTFLFMLFFVFYGGIVFGNLLIVITVVSDSHLHSPMYFLLANLSLIDLSLSSVTAPKMITDFFSQRKVISFKGCLVQ
MDGENHSVVSEFLFLGLTHSWEIQLLLLVFSSVLYVASITGNILIVFSVTTDPHLHSPMYFLLASLSFIDLGACSVTSPKMIYDLFRKRKVISFGGCIAQ


In [96]:
target_seq = homology['target']['align_seq'].replace('-', '')
print("target_seq:", target_seq)
source_seq = homology['source']['align_seq'].replace('-', '')
print("source_seq:", source_seq)


target_seq: MVTEFIFLGLSDSQGLQTFLFMLFFVFYGGIVFGNLLIVITVVSDSHLHSPMYFLLANLSLIDLSLSSVTAPKMITDFFSQRKVISFKGCLVQIFLLHFFGGSEMVILIAMGFDRYIAICKPLHYTTIMCGNACVGIMAVAWGIGFLHSVSQLAFAVHLPFCGPNEVDSFYCDLPRVIKLACTDTYRLDIMVIANSGVLTVCSFVLLIISYTIILMTIQHRPLDKSSKALSTLTAHITVVLLFFGPCVFIYAWPFPIKSLDKFLAVFYSVITPLLNPIIYTLRNKDMKTAIRQLRKWDAHSSVKF
source_seq: MDGENHSVVSEFLFLGLTHSWEIQLLLLVFSSVLYVASITGNILIVFSVTTDPHLHSPMYFLLASLSFIDLGACSVTSPKMIYDLFRKRKVISFGGCIAQIFFIHVVGGVEMVLLIAMAFDRYVALCKPLHYLTIMSPRMCLSFLAVAWTLGVSHSLFQLAFLVNLAFCGPNVLDSFYCDLPRLLRLACTDTYRLQFMVTVNSGFICVGTFFILLISYVFILFTVWKHSSGGSSKALSTLSAHSTVVLLFFGPPMFVYTRPHPNSQMDKFLAIFDAVLTPFLNPVVYTFRNKEMKAAIKRVCKQLVIYKRIS


In [118]:


ksize = 7

target_kmers = set(target_seq[i:i+ksize] for i in range(len(target_seq)-ksize+1))

In [119]:
source_kmers = set(source_seq[i:i+ksize] for i in range(len(source_seq)-ksize+1))

In [120]:
len(target_kmers)

299

In [121]:
len(source_kmers)

306

In [122]:
# target_kmers.intersection(source_kmers)

In [123]:
len(target_kmers.intersection(source_kmers))/min(len(source_kmers), len(target_kmers))

0.05351170568561873

In [125]:
ksizes = 2, 3, 4, 5, 6, 7, 8, 9

for ksize in ksizes:
    print(f"--- {ksize} ---")
    target_kmers = set(target_seq[i:i+ksize] for i in range(len(target_seq)-ksize+1))
    source_kmers = set(source_seq[i:i+ksize] for i in range(len(source_seq)-ksize+1))
    print("len(target_kmers):", len(target_kmers))
    print("len(source_kmers):", len(source_kmers))
    print("intersection:", len(target_kmers.intersection(source_kmers))/min(len(source_kmers), len(target_kmers)))

--- 2 ---
len(target_kmers): 181
len(source_kmers): 186
intersection: 0.7292817679558011
--- 3 ---
len(target_kmers): 287
len(source_kmers): 298
intersection: 0.31010452961672474
--- 4 ---
len(target_kmers): 301
len(source_kmers): 309
intersection: 0.16279069767441862
--- 5 ---
len(target_kmers): 301
len(source_kmers): 308
intersection: 0.10631229235880399
--- 6 ---
len(target_kmers): 300
len(source_kmers): 307
intersection: 0.07666666666666666
--- 7 ---
len(target_kmers): 299
len(source_kmers): 306
intersection: 0.05351170568561873
--- 8 ---
len(target_kmers): 298
len(source_kmers): 305
intersection: 0.03691275167785235
--- 9 ---
len(target_kmers): 297
len(source_kmers): 304
intersection: 0.020202020202020204


 | Amino acid    | Property              | Dayhoff |
 |---------------|-----------------------|---------|
 | C             | Sulfur polymerization | a       |
 | A, G, P, S, T | Small                 | b       |
 | D, E, N, Q    | Acid and amide        | c       |
 | H, K, R       | Basic                 | d       |
 | I, L, M, V    | Hydrophobic           | e       |
 | F, W, Y       | Aromatic              | f       |


In [133]:
dayhoff_mapping = {
    "C": "a",
    
    # Small
    "A": "b",
    "G": "b",
    "P": "b",
    "S": "b",
    "T": "b",
    
    # Acid and amide
    "D": "c",
    "E": "c",
    "N": "c",
    "Q": "c",
    
    # Basic
    "H": "d",
    "K": "d",
    "R": "d",
    
    # Hydrophobic
    "I": "e",
    "L": "e",
    "M": "e",
    "V": "e",
    
    # Aromatic
    "F": "f",
    "W": "f",
    "Y": "f"
}
dayhoff_mapping = str.maketrans(dayhoff_mapping)

# print(dayhoff_mapping)
target_dayhoff = target_seq.translate(dayhoff_mapping)
target_dayhoff

'eebcfefebebcbcbecbfefeeffeffbbeefbceeeeebeebcbdedbbeffeebcebeecebebbebbbdeebcffbcddeebfdbaeecefeedffbbbceeeeebebfcdfebeadbedfbbeeabcbaebeebebfbebfedbebcebfbedebfabbccecbffacebdeedebabcbfdeceeeebcbbeebeabfeeeeebfbeeeebecddbecdbbdbebbebbdebeeeeffbbaefefbfbfbedbecdfebeffbeebbeecbeefbedcdcedbbedceddfcbdbbedf'

In [134]:
source_dayhoff = source_seq.translate(dayhoff_mapping)

In [136]:
ksizes = 2, 3, 4, 5, 6, 7, 8, 9

for ksize in ksizes:
    print(f"--- {ksize} ---")
    target_kmers = set(target_dayhoff[i:i+ksize] for i in range(len(target_dayhoff)-ksize+1))
    source_kmers = set(source_dayhoff[i:i+ksize] for i in range(len(source_dayhoff)-ksize+1))
    print("len(target_kmers):", len(target_kmers))
    print("len(source_kmers):", len(source_kmers))
    print("intersection:", len(target_kmers.intersection(source_kmers))/min(len(source_kmers), len(target_kmers)))

--- 2 ---
len(target_kmers): 32
len(source_kmers): 32
intersection: 1.0
--- 3 ---
len(target_kmers): 110
len(source_kmers): 111
intersection: 0.8363636363636363
--- 4 ---
len(target_kmers): 204
len(source_kmers): 222
intersection: 0.5980392156862745
--- 5 ---
len(target_kmers): 268
len(source_kmers): 283
intersection: 0.417910447761194
--- 6 ---
len(target_kmers): 290
len(source_kmers): 301
intersection: 0.29310344827586204
--- 7 ---
len(target_kmers): 295
len(source_kmers): 305
intersection: 0.2305084745762712
--- 8 ---
len(target_kmers): 296
len(source_kmers): 305
intersection: 0.18243243243243243
--- 9 ---
len(target_kmers): 296
len(source_kmers): 304
intersection: 0.14864864864864866


In [124]:
homology.keys()

dict_keys(['method_link_type', 'target', 'type', 'dn_ds', 'taxonomy_level', 'source'])

In [84]:
ensembl_rest.overlap_translation("ENSP00000467301")

[{'align_type': None,
  'interpro': '',
  'id': 'SSF81321',
  'Parent': 'ENST00000585993',
  'type': 'SuperFamily',
  'description': '',
  'cigar_string': '',
  'feature_type': 'protein_feature',
  'translation_id': 1589091,
  'hseqname': 'SSF81321',
  'start': 2,
  'seq_region_name': 'ENSP00000467301',
  'hit_start': 0,
  'hit_end': 0,
  'end': 297},
 {'interpro': 'IPR000276',
  'align_type': None,
  'id': 'PS00237',
  'cigar_string': '',
  'feature_type': 'protein_feature',
  'translation_id': 1589091,
  'hseqname': 'PS00237',
  'type': 'Prosite_patterns',
  'Parent': 'ENST00000585993',
  'description': 'G protein-coupled receptor, rhodopsin-like',
  'start': 103,
  'end': 119,
  'hit_end': 0,
  'seq_region_name': 'ENSP00000467301',
  'hit_start': 0},
 {'interpro': '',
  'align_type': None,
  'id': 'TMhelix',
  'type': 'TMHMM',
  'Parent': 'ENST00000585993',
  'description': '',
  'feature_type': 'protein_feature',
  'hseqname': 'TMhelix',
  'translation_id': 1589091,
  'cigar_string

In [79]:
target_protein_id = homology['target']['protein_id']
source_protein_id = homology['source']['protein_id']

target_sig = protein_id_to_signature[target_protein_id]
source_sig = protein_id_to_signature[source_protein_id]



In [80]:
target_sig.similarity(source_sig)

0.0

In [ ]:
for homology in homologies:


In [28]:
ensembl_rest.lookup("ENST00000641515")

{'db_type': 'core',
 'source': 'havana',
 'object_type': 'Transcript',
 'biotype': 'protein_coding',
 'logic_name': 'havana',
 'assembly_name': 'GRCh38',
 'seq_region_name': '1',
 'id': 'ENST00000641515',
 'is_canonical': 0,
 'start': 65419,
 'end': 71585,
 'Parent': 'ENSG00000186092',
 'version': 2,
 'display_name': 'OR4F5-202',
 'species': 'homo_sapiens',
 'strand': 1}

In [36]:
# ensembl_rest.homology_ensemblgene("ENSG00000186092", target_species='mus_musculus', type='orthologues')

In [44]:
import requests, sys
from pprint import pprint
 
server = "https://rest.ensembl.org"
ext = "/homology/id/ENSG00000157764?target_species=mouse;type=orthologues"
 
r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
 
if not r.ok:
    r.raise_for_status()
    sys.exit()

decoded = r.json()
# print(repr(decoded))
pprint(decoded)

{'data': [{'homologies': [{'dn_ds': 0.04787,
                           'method_link_type': 'ENSEMBL_ORTHOLOGUES',
                           'source': {'align_seq': '-------------------MAALSGGGGGGAEPGQALFNGDMEPEAGAGAGAAASSAADPAIPEEVWNIKQMIKLTQEHIEALLDKFGGEHNPPSIYLEAYEEYTSKLDALQQREQQLL-ESLGNGTDFSVSSSASMDTVTSSSSSSLSVLPSSLSVFQNPTDVARSNPKSPQKPIVRVFLPNKQRTVVPARCGVTVRDSLKKALMMRGLIPECCAVYRIQDG-EKKPIGWDTDISWLTGEELHVEVLENVPLTTHNFVRKTFFTLAFCDFCRKLLFQGFRCQTCGYKFHQRCSTEVPLMCVNYDQLDLLFVSKFFEHHPIPQEEASLAETALTSGSSPSAPASDSIGPQILTSPSPSKSIPIPQPFRPADEDHRNQFGQRDRSSSAPNVHINTIEPVNIDD-------------LIRDQGFRGDG----------------------------------------GSTTGLSATPPASLPGSLTNVKALQKSPGPQRERKSSSSS--EDRNRMKTLGRRDSSDDWEIPDGQITVGQRIGSGSFGTVYKGKWHGDVAVKMLNVTAPTPQQLQAFKNEVGVLRKTRHVNILLFMGYSTKPQLAIVTQWCEGSSLYHHLHIIETKFEMIKLIDIARQTAQGMDYLHAKSIIHRDLKSNNIFLHEDLTVKIGDFGLATVKSRWSGSHQFEQLSGSILWMAPEVIRMQDKNPYSFQSDVYAFGIVLYELMTGQLPYSNINNRDQIIFMVGRGYLSPDLSKVRSNCPKAMKRLMAECLKKKRDERPLFPQILASIELLARSLPKIHRSASEPSLNRAGFQTEDFSLYACASPKTPIQA

In [ ]:
pprint()

In [ ]:
import requests, sys
 
server = "https://rest.ensembl.org"
ext = "/homology/id/ENSG00000157764?target_species=mouse;type=orthologues"
 
r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
 
if not r.ok:
    r.raise_for_status()
    sys.exit()

decoded = r.json()
print(repr(decoded))

In [37]:
import requests, sys
 
server = "http://rest.ensembl.org"
ext = "/homology/id/human/ENSG00000186092?format=condensed;target_species=mouse;type=orthologues"
 
r = requests.get(server+ext, headers={ "Content-Type" : "application/json"})
 
if not r.ok:
  r.raise_for_status()
  sys.exit()
 
decoded = r.json()
print(repr(decoded))
 

HTTPError: 404 Client Error: Not Found for url: http://rest.ensembl.org/homology/id/human/ENSG00000186092?format=condensed;target_species=mouse;type=orthologues

In [19]:
gene_id_with_orthology = 'ENSG00000198888'

In [20]:
ensembl_rest.lookup(gene_id_with_orthology)

{'biotype': 'protein_coding',
 'strand': 1,
 'display_name': 'MT-ND1',
 'version': 2,
 'species': 'homo_sapiens',
 'end': 4262,
 'object_type': 'Gene',
 'start': 3307,
 'source': 'insdc',
 'description': 'mitochondrially encoded NADH:ubiquinone oxidoreductase core subunit 1 [Source:HGNC Symbol;Acc:HGNC:7455]',
 'id': 'ENSG00000198888',
 'seq_region_name': 'MT',
 'db_type': 'core',
 'logic_name': 'mt_genbank_import',
 'assembly_name': 'GRCh38'}

In [25]:
ensembl_rest.overlap_region(gene_id_with_orthology)

HTTPError: Server returned HTTP status code: 404
Content: {'error': 'page not found. Please check your uri and refer to our documentation https://rest.ensembl.org/'}

In [21]:
ensembl_rest.xref_id(gene_id_with_orthology, all_levels=True)

[{'info_text': '',
  'version': '0',
  'display_id': 'ENSG00000198888',
  'description': None,
  'synonyms': [],
  'primary_id': 'ENSG00000198888',
  'info_type': 'DIRECT',
  'db_display_name': 'Expression Atlas',
  'dbname': 'ArrayExpress'},
 {'dbname': 'EntrezGene',
  'db_display_name': 'NCBI gene',
  'synonyms': [],
  'primary_id': '4535',
  'info_type': 'DEPENDENT',
  'description': 'NADH dehydrogenase, subunit 1 (complex I)',
  'info_text': '',
  'display_id': 'ND1',
  'version': '0'},
 {'display_id': 'MT-ND1',
  'version': '0',
  'info_text': 'Generated via ensembl_manual',
  'description': 'mitochondrially encoded NADH:ubiquinone oxidoreductase core subunit 1',
  'info_type': 'DIRECT',
  'synonyms': ['MTND1', 'NAD1', 'ND1'],
  'primary_id': 'HGNC:7455',
  'dbname': 'HGNC',
  'db_display_name': 'HGNC Symbol'},
 {'dbname': 'MIM_GENE',
  'db_display_name': 'MIM gene',
  'synonyms': [],
  'primary_id': '516000',
  'info_type': 'DEPENDENT',
  'description': 'COMPLEX I, SUBUNIT ND1; M

In [15]:
ensembl_rest.homology_ensemblgene(key)

{'data': []}

{'data': []}

In [25]:
orthologues = pd.read_csv("s3://kmer-hashing/hg38_mm38_protein_coding/hg38_mm38_ensembl_compara_orthologues.txt", sep='\t')
print(orthologues.shape)
orthologues.head()

(249084, 15)


,Gene stable ID,Transcript stable ID,Mouse gene stable ID,Mouse gene name,Mouse protein or transcript stable ID,Mouse homology type,%id. target Mouse gene identical to query gene,%id. query gene identical to target Mouse gene,Mouse Gene-order conservation score,Mouse Whole-genome alignment coverage,dS with Mouse,dN with Mouse,"Mouse orthology confidence [0 low, 1 high]",Last common ancestor with Mouse,Query protein or transcript ID
0,ENSG00000210049,ENST00000387314,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ENSG00000211459,ENST00000389680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ENSG00000210077,ENST00000387342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ENSG00000210082,ENST00000387347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ENSG00000209082,ENST00000386347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
